In [8]:
import cv2
import os
import numpy as np
from PIL import Image
from skimage.measure import label, regionprops_table

In [9]:
#Set paths accordingly
segmentations_path = "D:\\PRS_project\\Data_Set_HAM\\test\\melanoma\\segmentation\\"
img_output_path = "D:\\PRS_project\\Data_Set_HAM\\test\\melanoma\\center_cropped\\"
img_input_path = "D:\\PRS_project\\Data_Set_HAM\\test\\melanoma\\images\\"

In [10]:
#function to get the center coordinates of a single segmentation mask
def get_center_coordinates(seg_name):
    seg_path = segmentations_path + seg_name
    img = cv2.imread(seg_path)
    
    label_img = label(img)
    props = regionprops_table(label_img, properties=('centroid',))

    y_max = img.shape[0]
    x_max = img.shape[1]

    y_range = np.array([props['centroid-0'][0]-112,props['centroid-0'][0]+112])
    x_range = np.array([props['centroid-1'][0]-112,props['centroid-1'][0]+112])

    #if y_range is outside of the image, shift it back in
    if y_range[0] < 0:
        y_range = y_range - y_range[0]
    if y_range[1] > y_max:
        y_range = y_range - (y_range[1] - y_max)

    #if x_range is outside of the image, shift it back in
    if x_range[0] < 0:
        x_range = x_range - x_range[0]
    if x_range[1] > x_max:
        x_range = x_range - (x_range[1] - x_max)

    #join y_range and x_range into a 2D array, which will be stored in a list for all images in the folder
    center_px_range = np.array([y_range,x_range])
    key = seg_name.replace("_segmentation.png","")
    center_px_range_dict = {key:center_px_range}
    
    return center_px_range_dict

In [11]:
#getting the center coordinates of all the segmentation masks in the path and saving it in a dict
center_coordinate_ranges = {}

seg_list = os.listdir(segmentations_path)
for seg_name in seg_list:
    center_coordinate_ranges.update(get_center_coordinates(seg_name))

In [12]:
print(center_coordinate_ranges)

{'ISIC_0024351': array([[113.3383196 , 337.3383196 ],
       [191.36696783, 415.36696783]]), 'ISIC_0024400': array([[117.1679362 , 341.1679362 ],
       [196.30380245, 420.30380245]]), 'ISIC_0024449': array([[126.48245976, 350.48245976],
       [226.79879791, 450.79879791]]), 'ISIC_0024496': array([[120.90504934, 344.90504934],
       [174.74252144, 398.74252144]]), 'ISIC_0024516': array([[114.22807138, 338.22807138],
       [178.03501478, 402.03501478]]), 'ISIC_0024545': array([[113.7172191 , 337.7172191 ],
       [200.03060927, 424.03060927]]), 'ISIC_0024792': array([[ 88.33327804, 312.33327804],
       [197.63165546, 421.63165546]]), 'ISIC_0024806': array([[110.0149734 , 334.0149734 ],
       [182.97696746, 406.97696746]]), 'ISIC_0025018': array([[ 90.46396581, 314.46396581],
       [187.18514258, 411.18514258]]), 'ISIC_0025097': array([[125.02387602, 349.02387602],
       [141.70286832, 365.70286832]]), 'ISIC_0025145': array([[105.11917359, 329.11917359],
       [184.65680828, 408.

In [13]:
#function to crop the center 224x224 pixels of a single image and save it to the output folder
def center_crop(img_name):
    try:
        img_id = img_name.split('.')[0]
        
        if img_id in center_coordinate_ranges:
            left = center_coordinate_ranges[img_id][1][0]
            top = center_coordinate_ranges[img_id][0][0]
            right = center_coordinate_ranges[img_id][1][1]
            bottom = center_coordinate_ranges[img_id][0][1]
            
            img_path = img_input_path + img_name
            img = Image.open(img_path)
            
            # Crop image of above dimension (It will not change original image)
            img_cropped = img.crop((left, top, right, bottom))
            
            #save the cropped image to the output path, leaving the original image the same
            output_path = img_output_path + img_name
            img_cropped.save(output_path)
            
    except:
        print(f"Error cropping image {img_name}")

In [14]:
img_list = os.listdir(img_input_path)

for img_name in img_list:
    #crop the center 224x224 pixels
    center_cropped_img = center_crop(img_name)